In [1]:
import numpy as np
from sklearn.base import BaseEstimator

class LinearClassifier(BaseEstimator):
    """
    General class for binary linear classifiers. Implements the predict
    function, which is the same for all binary linear classifiers. There are
    also two utility functions.
    """

    def decision_function(self, X):
        """
        Computes the decision function for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """
        return X.dot(self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Select the positive or negative class label, depending on whether
        # the score was positive or negative.
        out = np.select([scores >= 0.0, scores < 0.0],
                        [self.positive_class,
                         self.negative_class])
        return out

    def find_classes(self, Y):
        """
        Finds the set of output classes in the output part Y of the training set.
        If there are exactly two classes, one of them is associated to positive
        classifier scores, the other one to negative scores. If the number of
        classes is not 2, an error is raised.
        """
        classes = sorted(set(Y))
        if len(classes) != 2:
            raise Exception("this does not seem to be a 2-class problem")
        self.positive_class = classes[1]
        self.negative_class = classes[0]

    def encode_outputs(self, Y):
        """
        A helper function that converts all outputs to +1 or -1.
        """
        return np.array([1 if y == self.positive_class else -1 for y in Y])

class Perceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_features = X.shape[1]
        self.w = np.zeros(n_features)

        # Perceptron algorithm:
        for i in range(self.n_iter):
            for x, y in zip(X, Ye):

                # Compute the output score for this instance.
                score = x.dot(self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    self.w += y*x


##### The following part is for the optional task.

### Sparse and dense vectors don't collaborate very well in NumPy/SciPy.
### Here are two utility functions that help us carry out some vector
### operations that we'll need.

def add_sparse_to_dense(x, w, factor):
    """
    Adds a sparse vector x, scaled by some factor, to a dense vector.
    This can be seen as the equivalent of w += factor * x when x is a dense
    vector.
    """
    w[x.indices] += factor * x.data

def sparse_dense_dot(x, w):
    """
    Computes the dot product between a sparse vector x and a dense vector w.
    """
    return np.dot(w[x.indices], x.data)


class SparsePerceptron(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm,
    assuming that the input feature matrix X is sparse.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.

        Note that this will only work if X is a sparse matrix, such as the
        output of a scikit-learn vectorizer.
        """
        self.find_classes(Y)

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector to all zeros.
        self.w = np.zeros(X.shape[1])

        # Iteration through sparse matrices can be a bit slow, so we first
        # prepare this list to speed up iteration.
        XY = list(zip(X, Ye))

        for i in range(self.n_iter):
            for x, y in XY:

                # Compute the output score for this instance.
                # (This corresponds to score = x.dot(self.w) above.)
                score = sparse_dense_dot(x, self.w)

                # If there was an error, update the weights.
                if y*score <= 0:
                    # (This corresponds to self.w += y*x above.)
                    add_sparse_to_dense(x, self.w, y)





In [2]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y


if __name__ == '__main__':
    
    # Read all the documents.
    X, Y = read_data('data/all_sentiment_shuffled.txt')
    
    # Split into training and test parts.
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                    random_state=0)

    # Set up the preprocessing steps and the classifier.
    pipeline = make_pipeline(
        TfidfVectorizer(),
        SelectKBest(k=1000),
        Normalizer(),

        # NB that this is our Perceptron, not sklearn.linear_model.Perceptron
        Perceptron()  
    )
    print("Classifier using our perceptron implementation")
    # Train the classifier.
    t0 = time.time()
    pipeline.fit(Xtrain, Ytrain)
    t1 = time.time()
    print('Training time: {:.2f} sec.'.format(t1-t0))

    # Evaluate on the test set.
    Yguess = pipeline.predict(Xtest)
    print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

Classifier using our perceptron implementation
Training time: 1.01 sec.
Accuracy: 0.7919.


In [3]:
class PegasusSVC(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the perceptron learning algorithm.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
    
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.lambda_ = 1/n_samples

        # Perceptron algorithm:
        for t in range(1,self.n_iter+1):

            idx = np.random.randint(n_samples)

            eta = 1/(self.lambda_*t)

            #for x, y in zip(X, Ye):
            x, y = X[idx], Ye[idx]

            # Compute the output score for this instance.
            score = self.w.dot(x)
            # If there was an error, update the weights.
            if y*score < 1:
                self.w = (1-eta*self.lambda_)*self.w + x*eta*y
            else:
                self.w *= 1-eta*self.lambda_
                

In [4]:
class PegasusLR(LinearClassifier):
    """
    A straightforward implementation of the perceptron learning algorithm.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter  = n_iter

#     def fit(self, X, Y):
#         """
#         Train a linear classifier using the perceptron learning algorithm.
#         """

#         # First determine which output class will be associated with positive
#         # and negative scores, respectively.
#         self.find_classes(Y)

#         # Convert all outputs to +1 (for the positive class) or -1 (negative).
#         Ye = self.encode_outputs(Y)

#         # If necessary, convert the sparse matrix returned by a vectorizer
#         # into a normal NumPy matrix.
#         if not isinstance(X, np.ndarray):
#             X = X.toarray()

#         # Initialize the weight vector to all zeros.
    
#         n_samples, n_features = X.shape
#         self.w = np.zeros(n_features)
#         self.lambda_ = 1/n_samples

#         # Perceptron algorithm:
#         for t in range(1,self.n_iter+1):

#             idx = np.random.randint(n_samples)

#             eta = 1/(self.lambda_*t)

#             #for x, y in zip(X, Ye):
#             x, y = X[idx], Ye[idx]
            
#             self.w = self.w * (1-eta*self.lambda_) - eta*y*x/(1+np.exp(y*(self.w*x)))
    def fit(self, X, Y):
        """
        Train a linear classifier using the logistic regression objective function.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.lambda_ = 1 / n_samples

        # Pegasos algorithm for logistic regression:
        for t in range(1, self.n_iter + 1):
            idx = np.random.randint(n_samples)
            x, y = X[idx], Ye[idx]

            eta = 1 / (self.lambda_ * t)

            # Compute the predicted probability
            pred_prob = 1 / (1 + np.exp(-y * np.dot(self.w, x)))

            # Update the weight vector
            self.w = (1 - eta * self.lambda_) * self.w + eta * y * x * pred_prob

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Apply the sigmoid function to convert scores to probabilities
        probabilities = 1 / (1 + np.exp(-scores))

        # Predict the class labels based on probabilities
        out = np.where(probabilities >= 0.5, self.positive_class, self.negative_class)
        return out

In [5]:
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# from aml_perceptron import Perceptron, SparsePerceptron

# This function reads the corpus, returns a list of documents, and a list
# of their corresponding polarity labels. 
def read_data(corpus_file):
    X = []
    Y = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            _, y, _, x = line.split(maxsplit=3)
            X.append(x.strip())
            Y.append(y)
    return X, Y



    
# Read all the documents.
#X, Y = read_data('DAT341-Applied-Machine-Learning/PA4/data/all_sentiment_shuffled.txt')
X, Y = read_data('data/all_sentiment_shuffled.txt')
# Split into training and test parts.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2,
                                                random_state=0)
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusSVC(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print("PegasosSVC")
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

PegasosSVC
Training time: 0.75 sec.
Accuracy: 0.8019.


In [6]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusLR(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print("PegasosLR")
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

/var/folders/l3/v953_kc921j6xpd2q4vytq940000gn/T/ipykernel_2273/393453158.py:77: RuntimeWarning: overflow encountered in exp
  pred_prob = 1 / (1 + np.exp(-y * np.dot(self.w, x)))


PegasosLR
Training time: 0.81 sec.
Accuracy: 0.5031.


/var/folders/l3/v953_kc921j6xpd2q4vytq940000gn/T/ipykernel_2273/393453158.py:93: RuntimeWarning: overflow encountered in exp
  probabilities = 1 / (1 + np.exp(-scores))


In [7]:
import numpy as np
from scipy.linalg.blas import ddot, dscal, daxpy


class PegasusLRBLAS(LinearClassifier):
    """
    A Logistic Regression classifier using Pegasos algorithm with BLAS functions for speedup.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the logistic regression objective function with BLAS.
        """

        # First determine which output class will be associated with positive
        # and negative scores, respectively.
        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # If necessary, convert the sparse matrix returned by a vectorizer
        # into a normal NumPy matrix.
        if not isinstance(X, np.ndarray):
            X = X.toarray()

        # Initialize the weight vector to all zeros.
        n_samples, n_features = X.shape
        self.w = np.zeros(n_features)
        self.lambda_ = 1 / n_samples

        # Pegasos algorithm for logistic regression with BLAS:
        for t in range(1, self.n_iter + 1):
            idx = np.random.randint(n_samples)
            x, y = X[idx], Ye[idx]

            eta = 1 / (self.lambda_ * t)

            # Compute dot product using ddot
            score = ddot(self.w, x)

            # Calculate predicted probability
            pred_prob = 1 / (1 + np.exp(-y * score))

            # Update weight vector using daxpy and dscal
            daxpy(self.w, x, a=eta * y * pred_prob)
            dscal(1 - eta * self.lambda_, self.w)

    def predict(self, X):
        """
        Predicts the outputs for the inputs X. The inputs are assumed to be
        stored in a matrix, where each row contains the features for one
        instance.
        """

        # First compute the output scores
        scores = self.decision_function(X)

        # Apply the sigmoid function to convert scores to probabilities
        probabilities = 1 / (1 + np.exp(-scores))

        # Predict the class labels based on probabilities
        out = np.where(probabilities >= 0.5, self.positive_class, self.negative_class)
        return out


In [8]:
# Set up the preprocessing steps and the classifier.
pipeline = make_pipeline(
    TfidfVectorizer(),
    SelectKBest(k=1000),
    Normalizer(),
    
    PegasusLRBLAS(n_iter=10000)
)
# Train the classifier.
t0 = time.time()
pipeline.fit(Xtrain, Ytrain)
t1 = time.time()
print("PegasosLRBLAS")
print('Training time: {:.2f} sec.'.format(t1-t0))
# Evaluate on the test set.
Yguess = pipeline.predict(Xtest)
print('Accuracy: {:.4f}.'.format(accuracy_score(Ytest, Yguess)))

PegasosLRBLAS
Training time: 0.78 sec.
Accuracy: 0.4964.


In [9]:
import numpy as np
from scipy.sparse import csr_matrix

class SparsePegasusLR(LinearClassifier):
    """
    A Logistic Regression classifier using Pegasos algorithm for sparse data.
    """

    def __init__(self, n_iter=20):
        """
        The constructor can optionally take a parameter n_iter specifying how
        many times we want to iterate through the training set.
        """
        self.n_iter = n_iter

    def fit(self, X, Y):
        """
        Train a linear classifier using the logistic regression objective function
        with sparse operations.

        X is assumed to be a sparse matrix (e.g., output of TfidfVectorizer).
        """

        self.find_classes(Y)

        # Convert all outputs to +1 (for the positive class) or -1 (negative).
        Ye = self.encode_outputs(Y)

        # Initialize the weight vector as a sparse csr matrix.
        self.w = csr_matrix((X.shape[1], 1))

        # Learning rate and regularization parameter
        self.lambda_ = 1 / X.shape[0]

        # Pegasos algorithm for logistic regression with sparse operations:
        for t in range(1, self.n_iter + 1):
          # Iterate through data points in sparse format
          for idx, (x, y) in enumerate(zip(X, Ye)):
            eta = 1 / (self.lambda_ * t)

            # Sparse dot product (optimized for sparse matrices)
            score = x.dot(self.w)

            # Calculate predicted probability
            pred_prob = 1 / (1 + np.exp(-y * score))

            # Sparse update of weight vector
            self.w += eta * y * x.multiply(pred_prob)

    def predict(self, X):
        """
        Predicts the outputs for the sparse input X.
        """

        scores = X.dot(self.w)
        probabilities = 1 / (1 + np.exp(-scores))
        out = np.where(probabilities >= 0.5, self.positive_class, self.negative_class)
        return out
